<a href="https://colab.research.google.com/github/ryansilvatrb/Sistema_de_Reconhecimento_Facial/blob/main/colab_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab: Access Webcam for Images and Video
This notebook will go through how to access and run code on images and video taken using your webcam.  

For this purpose of this tutorial we will be using OpenCV's Haar Cascade to do face detection on our Webcam image and video.

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import load_model

# ====================================
# CONFIGURAÇÕES
# ====================================
dataset_path = "dataset"
pessoas = ["Sheldon", "Penny", "Leonard", "Raj", "Amy", "Bernadette"]
img_size = 100

# ====================================
# PREPARAR DATASET
# ====================================
X, y = [], []

for idx, pessoa in enumerate(pessoas):
    folder = os.path.join(dataset_path, pessoa)
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (img_size, img_size))

        X.append(resized)
        y.append(idx)

X = np.array(X) / 255.0
X = np.expand_dims(X, axis=-1)
y = to_categorical(y, num_classes=len(pessoas))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ====================================
# DEFINIR MODELO CNN
# ====================================
model = Sequential([
    Input(shape=(img_size, img_size, 1)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(pessoas), activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# ====================================
# TREINAR MODELO
# ====================================
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Força CPU caso GPU dê erro

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=4)
loss, acc = model.evaluate(X_test, y_test)
print(f"Acurácia no teste: {acc*100:.2f}%")

# Salvar modelo
model.save("modelo_reconhecimento.h5")

# ====================================
# FUNÇÃO PARA DETECTAR E RECONHECER ROSTOS
# ====================================
def reconhecer_faces(imagem_path, modelo_path="modelo_reconhecimento.h5"):
    # Carregar modelo
    model = load_model(modelo_path)

    # Carregar classificador de rostos do OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Carregar imagem
    img = cv2.imread(imagem_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detectar rostos
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (img_size, img_size)) / 255.0
        face_resized = np.expand_dims(face_resized, axis=-1)
        face_resized = np.expand_dims(face_resized, axis=0)

        # Predição
        pred = model.predict(face_resized, verbose=0)
        idx = np.argmax(pred)
        nome = pessoas[idx]
        confianca = pred[0][idx]

        # Desenhar na imagem
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(img, f"{nome} ({confianca:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (255, 0, 0), 2)

    # Mostrar resultado
    cv2.imshow("Resultado", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# ====================================
# EXEMPLO DE USO
# ====================================
# Depois do treino, use assim para reconhecer em uma nova imagem:
# reconhecer_faces("foto_teste.jpg")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.1493 - loss: 2.7078 - val_accuracy: 0.0000e+00 - val_loss: 2.0695
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1493 - loss: 1.7989 - val_accuracy: 0.0000e+00 - val_loss: 1.9140
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1736 - loss: 1.7311 - val_accuracy: 0.0000e+00 - val_loss: 1.8594
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4479 - loss: 1.6570 - val_accuracy: 0.0000e+00 - val_loss: 1.8701
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6771 - loss: 1.5021 - val_accuracy: 0.0000e+00 - val_loss: 1.9044
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7083 - loss: 1.3122 - val_accuracy: 0.0000e+00 - val_loss: 2.0364
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6146 - loss: 1.3013 - val_accuracy: 0.0000e+00 - val_loss: 2.3220
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4722 - loss: 1.0990 - val_accuracy: 

Acurácia no teste: 0.00%
